## Cobot Projectile Obstacle Avoidance
Work Items:
1. Obstacle generator
2. YOLO image segmentation
3. Cobot control and DQN

In [1]:
import pybullet as p
import gym
import matplotlib.pyplot as plt
import numpy as np
import torch
import random
import cobot_ai4robotics

In [2]:
# Basic code to open the GUI environment window.
# https://www.gymlibrary.dev/content/environment_creation/ - make a custom gym environment.

env = gym.make("cobot_ai4robotics", apply_api_compatibility=True, renders=True, isDiscrete=False)
env.reset()

# Modify joint control so that it is a change to the current joint position, not specifying a joint position.
origin = [0,0,0,0,0,0]
while True:
    ob, reward, done, _, info = env.step(env.action_space.sample()) # Things will not move unless you call env.step.
    if done:
        break
    print(ob)
    # First 7 items in ob are the pose (or they're meant to be, they seem dodgy?). The next 4 numbers are the bounding box coordinates compressed to be from 0-1, and the last item is the distance to camera.
env.close()

c:\Users\mvsue\miniconda3\envs\Ai4Robotics_FinalProject\lib\site-packages\gym\utils\passive_env_checker.py:165: UserWarning: WARN: The obs returned by the `reset()` method is not within the observation space.
  logger.warn(f"{pre} is not within the observation space.")
c:\Users\mvsue\miniconda3\envs\Ai4Robotics_FinalProject\lib\site-packages\gym\utils\passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):
c:\Users\mvsue\miniconda3\envs\Ai4Robotics_FinalProject\lib\site-packages\gym\utils\passive_env_checker.py:137: UserWarning: WARN: The obs returned by the `step()` method was expecting a numpy array, actual type: <class 'list'>
  logger.warn(
c:\Users\mvsue\miniconda3\envs\Ai4Robotics_FinalProject\lib\site-packages\gym\spaces\box.py:227: UserWarning: WARN: Casting input x to numpy array.
  logger.warn("Casting input x to numpy array.")
c:\Users\mvsue\miniconda3\envs\Ai

[0.4358608400235886, 1.5727354065610004, -0.543840465178123, -1.709592559719885, -0.9859097363595244, -0.9757858123451123, -1.4351794789102992, -1, -1, -1, -1, -1]
[-0.6043683507247256, 1.5239260707130036, -2.565886608174668, 1.6995666898655928, -0.6679973421158547, -0.4929171076664225, -1.9635085386577007, -1, -1, -1, -1, -1]
[-1.2005523822533184, 0.2001611046989156, -1.9162299775733949, 2.6423974876618614, -1.9783657360013684, -0.1048444743700421, 0.28413304300185305, -1, -1, -1, -1, -1]
[1.2965023850313593, -2.51166220584551, 0.801397540068102, 1.7898588613367175, 0.3456390319767492, -0.7170171903120786, -1.2726537386486967, -1, -1, -1, -1, -1]
[2.1377545959057396, -1.8436068408049868, 0.8518663234333403, 1.7752222601355396, 1.0222493392683343, 0.18100151353335125, -0.21770626890733202, -1, -1, -1, -1, -1]
[0.8900855324630065, 0.7935059684096517, -0.6722156385939034, 3.866124976630488, 0.8933114633262271, -0.299964594359204, 0.43730217287784484, -1, -1, -1, -1, -1]
[-0.7964579747035